## [[Discourse] How do you pipe multiple arguments in a dask bag pipeline? #177](https://github.com/coiled/dask-community/issues/177)

In [ ]:
import dask.bag as db

In [ ]:
def download(x):
    return x + " data "

def process_data(x):
    if x == "test_none data ":
        return None
    else:
        return x + "processed data "

def save_for_url(x, y):
    return (x, y + "saved")

In [ ]:
# from functools import wraps

# def star(f):
#     @wraps(f)
#     def wrapper(*args, **kwargs):
#         return f(*args[0], *args[1:], **kwargs)
#     return wrapper

In [ ]:
urls = ['dask.org', 'docs.dask.org', 'exampels.dask.org', 'test_none']

In [ ]:
# TypeError: <lambda>() missing 1 required positional argument: 'data'

# (db
#  .from_sequence(urls)
#  .map(lambda url: (url, download(url)))
#  .map(lambda url, data: (url, process_data(data)))
#  .filter(lambda url, result: result is not None)
#  .map(lambda url, result: save_for_url(url, result))
#  ).compute()

In [ ]:
# Using `Bag.starmap()`

(db
 .from_sequence(urls)
 .map(lambda url: (url, download(url)))
 .starmap(lambda url, data: (url, process_data(data)))
 .filter(lambda x: x[1] is not None)
 .starmap(lambda url, result: save_for_url(url, result))
 ).compute()

In [ ]:
# With star() -- works

# (db
#  .from_sequence(urls)
#  .map(lambda url: (url, download(url)))
#  .map(star(lambda url, data: (url, process_data(data))))
#  .filter(star(lambda url, result: result is not None))
#  .map(star(lambda url, result: save_for_url(url, result)))
#  ).compute()

In [ ]:
# Using `Bag.zip()`

urls_bag = db.from_sequence(urls)

processed_urls_bag = (urls_bag
                      .map(lambda url: download(url))
                      .map(lambda data: process_data(data))
                      .filter(lambda result: result is not None)
                     )
db.zip(urls_bag, processed_urls_bag).starmap(lambda url, result: save_for_url(url, result)).compute()

In [ ]:
# Super-minimal reproducer

l = [("a", "b"), ("c", "d")]

l_bag = db.from_sequence(l)

# TypeError: <lambda>() missing 1 required positional argument: 'y'
l_bag.map(lambda x, y: x + y).compute()

# With start() -- works!
l_bag.map(star(lambda x, y: x + y)).compute()

l_bag.starmap(lambda x, y: x + y).compute()